## Day 20: Grove Positioning System

It's finally time to meet back up with the Elves. When you try to contact them, however, you get no reply. Perhaps you're out of range?

You know they're headed to the grove where the star fruit grows, so if you can figure out where that is, you should be able to meet back up with them.

Fortunately, your handheld device has a file (your puzzle input) that contains the grove's coordinates! Unfortunately, the file is encrypted - just in case the device were to fall into the wrong hands.

Maybe you can decrypt it?

When you were still back at the camp, you overheard some Elves talking about coordinate file encryption. The main operation involved in decrypting the file is called mixing.

The encrypted file is a list of numbers. To mix the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is circular, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.

For example, to move the 1 in a sequence like 4, 5, 6, 1, 7, 8, 9, the 1 moves one position forward: 4, 5, 6, 7, 1, 8, 9. To move the -2 in a sequence like 4, -2, 5, 6, 7, 8, 9, the -2 moves two positions backward, wrapping around: 4, 5, 6, 7, 8, -2, 9.

The numbers should be moved in the order they originally appear in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.

Consider this encrypted file:

```
1
2
-3
3
-2
0
4
```

Mixing this file proceeds as follows:

```
Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3:
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3:
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0:
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4:
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1:
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0:
1, 2, -3, 4, 0, 3, -2
```

Then, the grove coordinates can be found by looking at the **1000th**, **2000th**, and **3000th** numbers after the value `0`, wrapping around the list as necessary. In the above example, the 1000th number after `0` is `4`, the 2000th is `-3`, and the 3000th is `2`; adding these together produces **3**.

In [2]:
import pytest
from icecream import ic

In [4]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            yield int(line)

In [495]:
assert list(load_input('sample.txt')) == [
    1,
    2,
    -3,
    3,
    -2,
    -8,
    4,
]

In [542]:
def move(numbers: [int], num:int, tron=False) -> [int]:
    size = len(numbers)
    if num == 0:
        if tron:
            print(f"{num} does not move:")
            print(", ".join([str(n) for n in numbers]), end="\n\n")
        return numbers
    pos = numbers.index(num)
    if num > 0:
        if (pos + num) < size - 1:
            new_pos = (pos + num + 1) % size
        elif (pos + num) == size - 1:
            new_pos = size
        else:
            new_pos = (pos + num + 1) % size
    elif num < 0:
        if (pos + num) > 0:
            # ic('Caso A')
            new_pos = (pos + num) % size
        elif (pos + num) == 0:
            new_pos = size
        else:
            # ic('Caso B')
            new_pos = (size + (pos + num)) % size
        # ic(num, pos, new_pos)
    if tron:
        print(f"{num} moves between", end=' ') 
        if new_pos == 0:
            val_from = numbers[-1]
            val_to = numbers[0]
        elif new_pos == 1:
            val_from = numbers[0]
            val_to = numbers[1]
        elif new_pos == size - 1:
            val_from = numbers[-2]
            val_to = numbers[-1]
        elif new_pos == size:
            val_from = numbers[-1]
            val_to = numbers[0]
        else:
            val_from = numbers[new_pos-1]
            val_to = numbers[new_pos]
        print(f"{val_from} and {val_to} [{pos} -> {new_pos}] ")    
    if pos < new_pos:
        numbers.insert(new_pos, num)
        numbers.pop(pos)
    else:
        numbers.pop(pos)
        numbers.insert(new_pos, num)
    if tron:
        print(", ".join([str(n) for n in numbers]), end="\n\n")
    return numbers

In [543]:
# Testing bigger numbers

numbers = [1, 4, -6]
for num in numbers[:]:
    numbers = move(numbers, num, tron=True)
numbers

1 moves between 4 and -6 [0 -> 2] 
4, 1, -6

4 moves between 1 and -6 [0 -> 2] 
1, 4, -6

-6 moves between 4 and -6 [2 -> 2] 
1, 4, -6



[1, 4, -6]

In [544]:
# Testing negatives

assert move([1, 4, -2, 5, 6, 7, 8, 9], -2, tron=True) == [1, 4, 5, 6, 7, 8, 9, -2]

-2 moves between 9 and 1 [2 -> 8] 
1, 4, 5, 6, 7, 8, 9, -2



In [545]:
assert move([4, -2, 5, 6, 7, 8, 9], -2, tron=True) == [4, 5, 6, 7, 8, -2, 9]

-2 moves between 8 and 9 [1 -> 6] 
4, 5, 6, 7, 8, -2, 9



In [546]:
assert move([0, 1, 4, -2, 5, 6, 7, 8, 9], -2, tron=True) == [0, -2, 1, 4, 5, 6, 7, 8, 9]

-2 moves between 0 and 1 [3 -> 1] 
0, -2, 1, 4, 5, 6, 7, 8, 9



In [547]:
# Testing ppsitive numbers

assert move([4, 7, 5, 2, 3, 8, 9], 2, tron=True) == [4, 7, 5, 3, 8, 2, 9]

2 moves between 8 and 9 [3 -> 6] 
4, 7, 5, 3, 8, 2, 9



In [549]:
assert move([4, 9, 7, 5, 2, 3, 8], 2, tron=True) == [4, 9, 7, 5, 3, 8, 2]

2 moves between 8 and 4 [4 -> 7] 
4, 9, 7, 5, 3, 8, 2



In [550]:
assert move([4, 9, 7, 8, 5, 2, 3], 2, tron=True) == [4, 2, 9, 7, 8, 5, 3]

2 moves between 4 and 9 [5 -> 1] 
4, 2, 9, 7, 8, 5, 3



In [551]:
assert move([4, 9, 7, 8, 5, 3, 2], 2, tron=True) == [4, 9, 2, 7, 8, 5, 3]

2 moves between 9 and 7 [6 -> 2] 
4, 9, 2, 7, 8, 5, 3



In [552]:
from icecream import ic

numbers = list(load_input('sample.txt')) 
size = len(numbers)

print("Initial arrangement:")
print(", ".join([str(n) for n in numbers]), end="\n\n")
for num in numbers[:]:
    numbers = move(numbers, num, tron=True)

Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3 [0 -> 2] 
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3 [0 -> 3] 
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0 [1 -> 5] 
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4 [2 -> 6] 
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1 [2 -> 7] 
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0 [5 -> 3] 
1, 2, -3, 4, 0, 3, -2



In [558]:
numbers = [0, 1, 3]
for num in numbers[:]:
    numbers = move(numbers, num, tron=True) 
numbers

0 does not move:
0, 1, 3

1 moves between 3 and 0 [1 -> 3] 
0, 3, 1

3 moves between 3 and 1 [1 -> 2] 
0, 3, 1



[0, 3, 1]

In [554]:
def solution_one(source: Iterable, tron=False):
    numbers = list(source)
    size = len(numbers)
    if tron:
        print("Initial arrangement:")
        print(", ".join([str(n) for n in numbers]), end="\n\n")
    for num in numbers[:]:
        numbers = move(numbers, num, tron=tron)
    zero_pos = numbers.index(0) 
    after_1k = numbers[(zero_pos + 1000) % size]
    after_2k = numbers[(zero_pos + 2000) % size]
    after_3k = numbers[(zero_pos + 3000) % size]
    if tron:
        print(after_1k, after_2k, after_3k)
    return sum([after_1k, after_2k, after_3k])
    
    
assert solution_one(load_input('sample.txt'), tron=True) == 3

Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3 [0 -> 2] 
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3 [0 -> 3] 
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0 [1 -> 5] 
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4 [2 -> 6] 
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1 [2 -> 7] 
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0 [5 -> 3] 
1, 2, -3, 4, 0, 3, -2

4 -3 2


In [555]:
solution_one(load_input('test.txt'), tron=True)

Initial arrangement:
0, 1, 2, 3, 4, -3, -2, -1, -4

0 does not move:
0, 1, 2, 3, 4, -3, -2, -1, -4

1 moves between 2 and 3 [1 -> 3] 
0, 2, 1, 3, 4, -3, -2, -1, -4

2 moves between 3 and 4 [1 -> 4] 
0, 1, 3, 2, 4, -3, -2, -1, -4

3 moves between -3 and -2 [2 -> 6] 
0, 1, 2, 4, -3, 3, -2, -1, -4

4 moves between -1 and -4 [3 -> 8] 
0, 1, 2, -3, 3, -2, -1, 4, -4

-3 moves between -4 and 0 [3 -> 9] 
0, 1, 2, 3, -2, -1, 4, -4, -3

-2 moves between 1 and 2 [4 -> 2] 
0, 1, -2, 2, 3, -1, 4, -4, -3

-1 moves between 2 and 3 [5 -> 4] 
0, 1, -2, 2, -1, 3, 4, -4, -3

-4 moves between -2 and 2 [7 -> 3] 
0, 1, -2, -4, 2, -1, 3, 4, -3

1 -2 -4


-5

In [556]:
source = load_input('input.txt')    
numbers = list(source)
original = numbers[:]
size = len(numbers)
assert size == 5000
assert 0 in numbers
for num in numbers[:]:
    numbers = move(numbers, num)
assert size == 5000
print(numbers.index(0))
k1 = numbers[(1000 + numbers.index(0)) % size]
k2 = numbers[(2000 + numbers.index(0)) % size]
k3 = numbers[(3000 + numbers.index(0)) % size]
print(k1, k2, k3)
for n in original:
    assert n in numbers


4594
-4509 -6293 -8785


In [557]:
sol = solution_one(load_input('input.txt'), tron=False)
print(f"Solution part one: {sol}")

Solution part one: -19587


Mix your encrypted file exactly once. What is the sum of the three numbers that form the grove coordinates?

To begin, get your puzzle input.